# GWAS 1000 Genomes Project - Phase 1 

## Phenotype : Gender


24/01/2019

In [1]:
import time
t1= time.time()

### Hail environment + python packages 

In [2]:
import hail as hl
import hail.expr.aggregators as agg
hl.init()

Running on Apache Spark version 2.2.1
SparkUI available at http://10.142.0.5:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2-d33e2d1c19b2
LOGGING: writing to /home/hail/hail-20190205-1803-0.2-d33e2d1c19b2.log


In [3]:
import numpy as np
from bokeh.plotting import figure, output_file, show, save
from pprint import pprint
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import Span, ColumnDataSource
output_notebook()
from math import log, isnan, log10
from bokeh.models import *
from itertools import cycle
from hail.expr import aggregators
from hail.expr.expressions import *
from hail.expr.expressions import Expression
from hail.typecheck import *
from hail import Table
import hail

Loading BokehJS ...

In [4]:
from google.cloud import storage
storage.Client()
client = storage.Client()
import gcsfs
fs = gcsfs.GCSFileSystem(project='copd-genes')

No module named 'dask'


### Read the MT file 

In [5]:
# phase 1
mt = hl.read_matrix_table("gs://1k_genome/1000-genomes/VDS-of-all/ALL.chr.integrated_phase1_v3.20101123.snps_indels_svs.genotypes.mt")

### Annotate the MT file and select the phenotype of interest (here the gender)

table = (hl.import_table('gs://1k_genome/1000-genomes/other/sample_info/sample_info.csv', delimiter=',', missing='', quote='"').key_by('Sample'))

with fs.open('1k_genome/1000-genomes/other/sample_info/sample_info.csv',"rt") as f:
    data = pd.read_csv(f, na_values = str, header = 0)

key = data['Gender'].values

#create Label encoder for Gender
le = LabelEncoder()
le.fit(['male', 'female'])

In [6]:
#annotate MT file
table = (hl.import_table('gs://ines-work/KG-annotation-with-sexencoder.csv', delimiter=',', missing='', quote='"',types={'Gender_Classification':hl.tfloat64}).key_by('Sample'))
mt = mt.annotate_cols(**table[mt.s])

2019-02-05 18:03:24 Hail: INFO: Reading table with no type imputation
  Loading column 'Sample' as type 'str' (type not specified)
  Loading column 'Family_ID' as type 'str' (type not specified)
  Loading column 'Population' as type 'str' (type not specified)
  Loading column 'Population_Description' as type 'str' (type not specified)
  Loading column 'Gender' as type 'str' (type not specified)
  Loading column 'Relationship' as type 'str' (type not specified)
  Loading column 'Unexpected_Parent_Child' as type 'str' (type not specified)
  Loading column 'Non_Paternity' as type 'str' (type not specified)
  Loading column 'Siblings' as type 'str' (type not specified)
  Loading column 'Grandparents' as type 'str' (type not specified)
  Loading column 'Avuncular' as type 'str' (type not specified)
  Loading column 'Half_Siblings' as type 'str' (type not specified)
  Loading column 'Unknown_Second_Order' as type 'str' (type not specified)
  Loading column 'Third_Order' as type 'str' (type n

In [7]:
mt.aggregate_cols(agg.counter(mt.Gender))

2019-02-05 18:03:31 Hail: INFO: Coerced sorted dataset
2019-02-05 18:03:34 Hail: INFO: Coerced sorted dataset


{'female': 567, 'male': 525}

### Take only common variants 

In [8]:
#filter MAF
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)
#print(mt.count()) (13404583, 1092)

#filter only SNPs
mt=mt.filter_rows(hl.is_snp(mt.alleles[0], mt.alleles[1]))
#print(mt.count()) (12194564, 1092)

### PCA

In [9]:
eigenvalues, scores, loadings = hl.hwe_normalized_pca(mt.GT, k=2)

2019-02-05 18:04:48 Hail: INFO: hwe_normalized_pca: running PCA using 12176620 variants.
2019-02-05 18:05:58 Hail: INFO: pca: running PCA with 2 components...
2019-02-05 18:08:14 Hail: INFO: Coerced sorted dataset


In [10]:
mt = mt.annotate_cols(pca = scores[mt.s])

In [16]:
p = hl.plot.scatter(scores.scores[0], scores.scores[1],
                    label= mt.cols()[scores.s].Super_Population,
                    title='PCA on Phase 1 of 1KG', xlabel='PC1', ylabel='PC2')
show(p)

2019-02-05 21:02:50 Hail: INFO: Coerced sorted dataset
2019-02-05 21:02:53 Hail: INFO: Coerced sorted dataset


In [12]:
mt = mt.annotate_cols(pca = scores[mt.s])

### Logistic + pca covariate

In [13]:
gwas = hl.logistic_regression_rows(
           test='wald',
           y=mt.col.Gender_Classification,
           x=mt.GT.n_alt_alleles(),
           covariates=[1, mt.pca.scores[0], mt.pca.scores[1]])

2019-02-05 18:08:24 Hail: INFO: Coerced sorted dataset
2019-02-05 18:08:29 Hail: INFO: logistic_regression_rows: running wald on 1092 samples for response variable y,
    with input variable x, and 3 additional covariates...


### Manhattan plot 

In [14]:
manhattan = hl.plot.manhattan(gwas.p_value, title='Manhattan Plot on Phase 1 of 1KG')
manhattan.legend.label_text_font_size = '15pt'
manhattan.xaxis.axis_label_text_font_size = "25pt"
manhattan.yaxis.axis_label_text_font_size = "25pt"
show(manhattan)

In [15]:
t2= time.time()
(t2-t1) / 60

7.518219272295634